# Beyond Wrapped: Spotify Analytics - Phase 1

**Course**: IME 565 - Predictive Data Analytics for Engineers  
**Team**: Nicolo DiFerdinando, Joe Mascher, Rithvik Shetty  
**Phase**: Foundation Analytics

---

## Objectives
1. Load and explore Spotify track datasets
2. Perform comprehensive data cleaning and preprocessing
3. Analyze audio features and their distributions
4. Identify top tracks, artists, and genres
5. Create temporal analysis (if timestamp data available)
6. Build visualizations for insights

## 1. Setup and Imports

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Statistics
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Utilities
import os
import warnings
from pathlib import Path

# Settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("All packages imported successfully!")

## 2. Data Loading

This section auto-detects CSV files in the data directory and loads them.

In [ ]:
# Find all CSV files in data directory
data_dir = Path('data')
csv_files = list(data_dir.glob('*.csv'))

print(f"Found {len(csv_files)} CSV files in data directory:")
for i, file in enumerate(csv_files, 1):
    file_size = file.stat().st_size / (1024 * 1024)
    print(f"  {i}. {file.name} ({file_size:.2f} MB)")

# Load the first CSV file
if csv_files:
    selected_file = csv_files[0]
    print(f"\nLoading: {selected_file.name}...")
    
    df = pd.read_csv(selected_file)
    print(f"Loaded {len(df):,} rows and {len(df.columns)} columns")
else:
    print("\nNo CSV files found in data/ directory!")
    print("Please download datasets from Kaggle (see data/README.md)")

In [ ]:
# Display first few rows
print("Dataset Preview:")
df.head(10)

In [ ]:
# Dataset info
print("Dataset Information:")
print("=" * 50)
df.info()

In [ ]:
# Column names
print("\nColumn Names:")
print(df.columns.tolist())

## 3. Data Understanding and Exploration

In [ ]:
# Basic statistics
print("Basic Statistics:")
df.describe()

In [ ]:
# Check for missing values
print("Missing Values Analysis:")
print("=" * 50)
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing Count': missing.values,
    'Percentage': missing_pct.values
}).sort_values('Missing Count', ascending=False)

missing_df[missing_df['Missing Count'] > 0]

In [ ]:
# Identify audio feature columns
audio_features = ['danceability', 'energy', 'loudness', 'speechiness', 
                  'acousticness', 'instrumentalness', 'liveness', 'valence', 
                  'tempo', 'duration_ms', 'key', 'mode', 'time_signature']

# Find which audio features exist in this dataset
available_features = [col for col in audio_features if col in df.columns]

print(f"Available Audio Features ({len(available_features)}):")
for feature in available_features:
    print(f"  - {feature}")

if available_features:
    print(f"\nFound {len(available_features)} audio features to analyze")
else:
    print("\nNo standard audio features found.")

## 4. Data Cleaning and Preprocessing

In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

print(f"Original dataset: {len(df_clean):,} rows")

# Remove duplicates
initial_count = len(df_clean)
df_clean = df_clean.drop_duplicates()
duplicates_removed = initial_count - len(df_clean)
print(f"Removed {duplicates_removed:,} duplicate rows")

# Handle missing values in audio features
if available_features:
    initial_count = len(df_clean)
    df_clean = df_clean.dropna(subset=available_features)
    missing_removed = initial_count - len(df_clean)
    print(f"Removed {missing_removed:,} rows with missing audio features")

# Remove invalid loudness values (should be negative dB)
if 'loudness' in df_clean.columns:
    initial_count = len(df_clean)
    df_clean = df_clean[df_clean['loudness'] <= 0]
    invalid_removed = initial_count - len(df_clean)
    print(f"Removed {invalid_removed:,} rows with invalid loudness values")

# Remove extremely short durations
if 'duration_ms' in df_clean.columns:
    initial_count = len(df_clean)
    df_clean = df_clean[df_clean['duration_ms'] >= 5000]
    short_removed = initial_count - len(df_clean)
    print(f"Removed {short_removed:,} rows with duration < 5 seconds")

print(f"\nFinal cleaned dataset: {len(df_clean):,} rows")

## 5. Audio Feature Analysis

In [ ]:
# Distribution of audio features
if available_features:
    normalized_features = [f for f in available_features 
                          if f not in ['loudness', 'tempo', 'duration_ms', 'key', 'mode', 'time_signature']]
    
    if normalized_features:
        fig, axes = plt.subplots(3, 3, figsize=(15, 12))
        axes = axes.flatten()
        
        for idx, feature in enumerate(normalized_features[:9]):
            axes[idx].hist(df_clean[feature].dropna(), bins=50, edgecolor='black', alpha=0.7)
            axes[idx].set_title(f'{feature.capitalize()} Distribution', fontsize=12, fontweight='bold')
            axes[idx].set_xlabel(feature.capitalize())
            axes[idx].set_ylabel('Frequency')
            axes[idx].grid(True, alpha=0.3)
            
            mean_val = df_clean[feature].mean()
            axes[idx].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.3f}')
            axes[idx].legend()
        
        for idx in range(len(normalized_features), 9):
            axes[idx].set_visible(False)
        
        plt.tight_layout()
        plt.suptitle('Audio Feature Distributions', fontsize=16, fontweight='bold', y=1.001)
        plt.show()

In [ ]:
# Correlation matrix of audio features
if len(available_features) > 1:
    plt.figure(figsize=(12, 10))
    correlation_matrix = df_clean[available_features].corr()
    
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    
    plt.title('Audio Features Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()

## 6. Feature Engineering: Composite Metrics

In [ ]:
# Create composite features

# Mood Score
if all(col in df_clean.columns for col in ['valence', 'energy', 'acousticness']):
    df_clean['mood_score'] = (
        0.5 * df_clean['valence'] + 
        0.3 * df_clean['energy'] + 
        0.2 * (1 - df_clean['acousticness'])
    )
    print("Created 'mood_score'")

# Grooviness Index
if all(col in df_clean.columns for col in ['danceability', 'energy', 'tempo']):
    tempo_normalized = (df_clean['tempo'] - df_clean['tempo'].min()) / (df_clean['tempo'].max() - df_clean['tempo'].min())
    df_clean['grooviness'] = (
        0.4 * df_clean['danceability'] + 
        0.3 * df_clean['energy'] + 
        0.3 * tempo_normalized
    )
    print("Created 'grooviness'")

# Focus Score
if all(col in df_clean.columns for col in ['speechiness', 'energy', 'instrumentalness']):
    df_clean['focus_score'] = (
        0.4 * (1 - df_clean['speechiness']) + 
        0.3 * df_clean['instrumentalness'] + 
        0.3 * (1 - abs(df_clean['energy'] - 0.5) * 2)
    )
    print("Created 'focus_score'")

# Relaxation Score
if all(col in df_clean.columns for col in ['energy', 'acousticness', 'tempo']):
    tempo_normalized = (df_clean['tempo'] - df_clean['tempo'].min()) / (df_clean['tempo'].max() - df_clean['tempo'].min())
    df_clean['relaxation_score'] = (
        0.4 * (1 - df_clean['energy']) + 
        0.3 * df_clean['acousticness'] + 
        0.3 * (1 - tempo_normalized)
    )
    print("Created 'relaxation_score'")

print("\nComposite features created successfully!")

## 7. Top Tracks, Artists, and Genres Analysis

In [ ]:
# Identify column names
track_col = None
artist_col = None
genre_col = None
popularity_col = None

for col in df_clean.columns:
    col_lower = col.lower()
    if 'track' in col_lower and 'name' in col_lower:
        track_col = col
    elif 'artist' in col_lower and ('name' in col_lower or col_lower == 'artists'):
        artist_col = col
    elif 'genre' in col_lower:
        genre_col = col
    elif 'popular' in col_lower:
        popularity_col = col

print("Identified columns:")
print(f"  Track: {track_col}")
print(f"  Artist: {artist_col}")
print(f"  Genre: {genre_col}")
print(f"  Popularity: {popularity_col}")

In [ ]:
# Top Artists
if artist_col:
    print("Top 20 Artists:")
    print("=" * 50)
    top_artists = df_clean[artist_col].value_counts().head(20)
    
    plt.figure(figsize=(12, 8))
    top_artists.plot(kind='barh', color='skyblue', edgecolor='black')
    plt.xlabel('Number of Tracks', fontsize=12)
    plt.ylabel('Artist', fontsize=12)
    plt.title('Top 20 Artists by Track Count', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print(top_artists)
else:
    print("Artist column not found")

In [ ]:
# Top Genres
if genre_col:
    print("Top 20 Genres:")
    print("=" * 50)
    top_genres = df_clean[genre_col].value_counts().head(20)
    
    plt.figure(figsize=(12, 8))
    top_genres.plot(kind='bar', color='lightcoral', edgecolor='black')
    plt.xlabel('Genre', fontsize=12)
    plt.ylabel('Number of Tracks', fontsize=12)
    plt.title('Top 20 Genres by Track Count', fontsize=14, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(top_genres)
else:
    print("Genre column not found")

## 8. Context Classification

In [ ]:
# Rule-based context classification
def classify_context(row):
    if row.get('energy', 0) > 0.7 and row.get('danceability', 0) > 0.6:
        return 'Workout'
    if row.get('speechiness', 0) < 0.2 and row.get('instrumentalness', 0) > 0.5:
        return 'Focus'
    if row.get('energy', 0) < 0.4 and row.get('acousticness', 0) > 0.5:
        return 'Relaxation'
    if row.get('valence', 0) > 0.6 and row.get('energy', 0) > 0.6 and row.get('danceability', 0) > 0.6:
        return 'Party'
    return 'General'

if available_features:
    df_clean['context'] = df_clean.apply(classify_context, axis=1)
    
    print("Context Classification Results:")
    print("=" * 50)
    context_counts = df_clean['context'].value_counts()
    print(context_counts)
    
    plt.figure(figsize=(10, 6))
    context_counts.plot(kind='bar', color='teal', edgecolor='black', alpha=0.7)
    plt.xlabel('Context', fontsize=12)
    plt.ylabel('Number of Tracks', fontsize=12)
    plt.title('Track Distribution by Listening Context', fontsize=14, fontweight='bold')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

## 9. Export Processed Data

In [ ]:
# Save cleaned dataset
output_path = Path('data/processed_spotify_data.csv')
df_clean.to_csv(output_path, index=False)

print(f"Processed data saved to: {output_path}")
print(f"  Rows: {len(df_clean):,}")
print(f"  Columns: {len(df_clean.columns)}")

## 10. Summary

In [ ]:
print("=" * 70)
print("PHASE 1 ANALYSIS SUMMARY")
print("=" * 70)

print(f"\nDataset Overview:")
print(f"  Total tracks analyzed: {len(df_clean):,}")
print(f"  Total features: {len(df_clean.columns)}")
print(f"  Audio features: {len(available_features)}")

if artist_col:
    print(f"\nArtists:")
    print(f"  Unique artists: {df_clean[artist_col].nunique():,}")

if genre_col:
    print(f"\nGenres:")
    print(f"  Unique genres: {df_clean[genre_col].nunique():,}")

if 'energy' in df_clean.columns:
    print(f"\nAudio Characteristics:")
    print(f"  Average energy: {df_clean['energy'].mean():.3f}")
    if 'valence' in df_clean.columns:
        print(f"  Average valence: {df_clean['valence'].mean():.3f}")
    if 'danceability' in df_clean.columns:
        print(f"  Average danceability: {df_clean['danceability'].mean():.3f}")

print(f"\nPhase 1 Analysis Complete!")
print("=" * 70)